In [1]:
# Imports
import pandas as pd
import math
from functools import reduce
import operator

In [2]:
# Opening the vigenere.txt
with open('vigenere.txt', 'r', encoding='utf-8-sig') as file:
    coded_msg = file.read().replace('\n', '')
display(coded_msg)

'MYHSIFPFGIMUCEXIPRKHFFQPRVAGIDDVKVRXECSKAPFGHMESJWUSSEHNEZIXFFLPQDVTCEUGTEEMFRQXWYCLPPAMBSKSTTPGSMIDNSESZJBDWJWSPQYINUVRFXPVPCEOZQRBNLUIINSRPXLEEHKSTTPGCEIMCSKVVVTJQRBSIUCKJOIIXXOVHYEFLINOEXFDPZJVFKTETVFXTTVJVRTBXRVGJRAIFPSRGTDXYSIWYXWVFPAQSSEHNEZIXFVRXQPRURVWBXWVCEIMCSKVVVUCXYWJAAGPUHYIDTMJFFSYUSISMIDNSESRRPILVUFSPTEIHYMEGMTVRRPREEDISHXHVTFVQKIIMFRQILVKRCAUPZTVGMCFVTIIQPRUPVEGIMWICFGIAVVRZQASJHKLQLEPUIIQSLRGGSUHSESUQQCWJCLPEWEJPRVDXGRRVHFWINCIPPLMKVYEFTLRGXSAHIJHVTBTHLGZRFDQZGVVKPRUPCSASWYSUAQWEMSUIHTPFDVHEEIVRSYITLRJVWTJXFIIWQAZVGZRYPGYWEIDNXYOKKUKIJOSYZSEEQVLMHPVTKYEXRNOEXAJVBBFAXTHXSYEEBEUSLWONRZQRPAJVTZVZQGRVGJLMGHRBUYZZMERNIFWMEYKSABYTVRRPUIVZKSAAMKHCIYDVVHYEZBETVZRQGCNSEIQSLLARRUICDCIIFWEEQCIHTVESJWITRVSUOUCHESJWMCHXSEXXTRVGJAUILFIKXTTWVELEXXXZSJPUUINWCPNTZZCCIZIEERRPXLMCZSIXDWKHYIMTVFDCEZTEERKLQGEUWFLMKISFFYSWXLGTPAHIIHFKQILVFKLQKIIMEEFJVVCWXTTWVWEZQCXZCEWOGMVGFYFUSIHYISDSUBVWEXRDSEGDXIJCLXRDVLBZZQGWRZSVAILVFYSASJFFKLQJRZHPSRJWRZCIHTRECNQKKSZQVMEGIRQYMZVQZZCMACWKVISGVLFIKXTTAF

In [3]:
''' Alphabet dictionary for the decipher process '''
alphabet = {
    "A": 0,"B": 1,"C": 2,"D": 3,"E": 4,"F": 5,"G": 6,"H": 7,"I": 8,"J": 9,"K": 10,"L": 11,"M": 12,"N": 13, 
    "O": 14,"P": 15,"Q": 16,"R": 17,"S": 18,"T": 19,"U": 20,"V": 21,"W": 22,"X": 23,"Y": 24,"Z": 25
}
reverse_alphabet = {v: k for k, v in alphabet.items()}

In [4]:
''' Kasiski attack for letter frequency '''
def kasiski_attack(text, n):
    i = 0
    count = {}
    
    for i in range(0, len(text)-(n-1)):
        s = text[i]
        for j in range(i+1, i+n):
            if j < len(text):
                s = s + text[j]
        if s in count:
            count[s] += 1
        else:
            count[s] = 1
        
    return count

In [5]:
def find_gcd(list):
    x = reduce(math.gcd, list)
    return x

In [6]:
''' Custom substraction for a system with only numbers in ranger [0,25] '''
def cyclic_subtraction(a,b):
    temp = a - b
    if temp < 0:
        temp = 26 + temp
    return temp

In [7]:
''' Anayzing the decrypted text '''
columns = ['Length','Substring', 'Frequency']
df = pd.DataFrame(columns = columns)

# All substrings with length>=4 and frequency>=3
for q in range(4,10):
    count = kasiski_attack(coded_msg, q)
    for key in count:
        if count[key] > 2:
            df = df.append({'Length': q, 'Substring': key, 'Frequency': count[key]}, ignore_index=True)
            #print ("Length:", q, " substring:", key, " freq:", count[key])

# Find the distances and gcd and adding them to the dataframe
la = [[] for _ in range(52)]
for index, row in df.iterrows():
    count_stars = 0
    count = 0
    start = False
    for char in coded_msg.replace(row['Substring'], "*"):
        if char == "*":
            if count != 0:
                la[index].append(count-1)
            count_stars = count_stars + 1
            start = True
            count=0
        if count_stars == row['Frequency']:
            break
        if start == True:
            count = count + 1
            
df['Distances'] = la

gcd_list = []
for index, row in df.iterrows():
    gcd_list.append(find_gcd(row['Distances']))
    
df['GCD'] = gcd_list
#display(df)

In [8]:
# Narrowing the above results according to gcd
indexNames = df[df['GCD'] < 3 ].index

df.drop(indexNames , inplace=True)
df.reset_index()

,index,Length,Substring,Frequency,Distances,GCD
0,10,4,QKII,3,"[500, 1445]",5
1,16,4,FKLQ,3,"[87, 339]",3
2,32,5,IKXTT,3,"[240, 1332]",12
3,33,5,KLQJR,3,"[765, 9]",9
4,46,7,KVVTQDT,4,"[14, 574, 441]",7
5,47,7,VVTQDTC,4,"[14, 574, 441]",7
6,48,7,VTQDTCS,4,"[14, 574, 441]",7


In [9]:
n=7
#https://stackoverflow.com/questions/22571259/split-a-string-into-n-equal-parts
parts = [coded_msg[i:i+n] for i in range(0, len(coded_msg), n)] # Split the message into n equal parts
#display(parts)
mapping = [list(word) for word in parts] # For each part we create a list with the letters of it
display(mapping)

[['M', 'Y', 'H', 'S', 'I', 'F', 'P'],
 ['F', 'G', 'I', 'M', 'U', 'C', 'E'],
 ['X', 'I', 'P', 'R', 'K', 'H', 'F'],
 ['F', 'Q', 'P', 'R', 'V', 'A', 'G'],
 ['I', 'D', 'D', 'V', 'K', 'V', 'R'],
 ['X', 'E', 'C', 'S', 'K', 'A', 'P'],
 ['F', 'G', 'H', 'M', 'E', 'S', 'J'],
 ['W', 'U', 'S', 'S', 'E', 'H', 'N'],
 ['E', 'Z', 'I', 'X', 'F', 'F', 'L'],
 ['P', 'Q', 'D', 'V', 'T', 'C', 'E'],
 ['U', 'G', 'T', 'E', 'E', 'M', 'F'],
 ['R', 'Q', 'X', 'W', 'Y', 'C', 'L'],
 ['P', 'P', 'A', 'M', 'B', 'S', 'K'],
 ['S', 'T', 'T', 'P', 'G', 'S', 'M'],
 ['I', 'D', 'N', 'S', 'E', 'S', 'Z'],
 ['J', 'B', 'D', 'W', 'J', 'W', 'S'],
 ['P', 'Q', 'Y', 'I', 'N', 'U', 'V'],
 ['R', 'F', 'X', 'P', 'V', 'P', 'C'],
 ['E', 'O', 'Z', 'Q', 'R', 'B', 'N'],
 ['L', 'U', 'I', 'I', 'N', 'S', 'R'],
 ['P', 'X', 'L', 'E', 'E', 'H', 'K'],
 ['S', 'T', 'T', 'P', 'G', 'C', 'E'],
 ['I', 'M', 'C', 'S', 'K', 'V', 'V'],
 ['V', 'T', 'J', 'Q', 'R', 'B', 'S'],
 ['I', 'U', 'C', 'K', 'J', 'O', 'I'],
 ['I', 'X', 'X', 'O', 'V', 'H', 'Y'],
 ['E', 'F', 

In [10]:
# Creating n-buckets with letters from the columns of mapping
df_list = []
columns = ['Letter', 'Frequency']
df_letters = pd.DataFrame(columns = columns)
for j in range(0,n):
    letters = []
    for i in range(0,len(mapping)):
        letters.append(mapping[i][j])
    text = ''.join(map(str, letters))
    print(str(j))
    print(text)
    count = kasiski_attack(text, 1)
    #print(count)
    df_list.append(count)

0
MFXFIXFWEPURPSIJPRELPSIVIIEXFXTRGWAEXWIVAISIRSMRSVRCGIEFZLISQPVHPESTRVSAIHYWWRIKYLYXAEWPZLYISRSIERIRIIWOWXAXEPPIPIIELLYPKLEXZWYIWGXZVSLSIQMMMSXHRTQRXHXWLWXMRRIILWXVIMSYWKIRIECRWXAWXMILJTMVTRKMXIMMXYWISIPAQVILALAIVHXYXCRHKVXYEGRGREIYQWRPILPYMIYXLILIIVMXSVJVILVOVLSMRSSTYQYTESTGEXVXRXPLXLJFYIJIZXLJEWXIYXXAHSXZLAXZEEKVLWWWLIIIIJLMRPXIPXPTSJJLMWSISAMVSAIERRWSHSHGRIPPEWVRICP
1
YGIQDEGUZQGQPTDBQFOUXTMTUXFFKTBATYQZQBMUADYDPPEPHQQAMQGGQQQUQEDFPFABFKAQHEITQYDUZMEAXEOAQMZFAPAYZQQUFHIUMXUTXUNZXXMZQMSAQQFTQOFSEDRQAAQRHKEZAGTYEMQMXDUKMYGYUFFIQMPFDEZFEDNQZDABEQUMMQBQDQXZQPMQKHEQSDFEPKMTQXXMTUTQUUDXFAZQUEAZXTITPOMMMKAQYMAYZMPQQPQPGMQRDKAFEOAQUQYEYZDAFQKQHIADOTFQQTQQAUDQXFQDQTQPOGBFZGRIQDIQMAIQRZQUQQQNMZDKKGMEQXAFHTMQIUGMERRIPUADPUPFFORDAZIUPEQXBEEMYXX
2
HIPPDCHSIDTXATNDYXZILTCJCXLDTTXIDXSIPXCCGTUNITGRXKIUCPIIALSHCWXWLTHTDPSWTETJAPNKSHXJTBNJGGZWBUADBGSIWTTCCTITXUTILDTTGKWHIKJTCGUDXXDGISJJTKGVCVTXDXCCTTBHITGECEDWPNTWNCJIXTXHLIUGIDACIPDNDDAIDHHALTWGXHDBTDKDCXAIPCPAATTXAJDGKTICBXXXHWGGRDINTKKPNGDDJWJPCA

In [11]:
# Code snippet just for testing reasons
for i in range(0, n):
    print(i)
    print(df_list[i])

0
{'M': 18, 'F': 6, 'X': 36, 'I': 52, 'W': 25, 'E': 21, 'P': 21, 'U': 1, 'R': 29, 'S': 27, 'J': 9, 'L': 26, 'V': 20, 'T': 9, 'G': 7, 'A': 13, 'C': 4, 'Z': 7, 'Q': 6, 'H': 9, 'Y': 17, 'K': 6, 'O': 2}
1
{'Y': 13, 'G': 12, 'I': 12, 'Q': 64, 'D': 23, 'E': 22, 'U': 24, 'Z': 19, 'P': 18, 'T': 18, 'B': 8, 'F': 24, 'O': 8, 'X': 18, 'M': 29, 'K': 11, 'A': 27, 'H': 7, 'N': 3, 'S': 3, 'R': 8}
2
{'H': 18, 'I': 32, 'P': 20, 'D': 39, 'C': 25, 'S': 10, 'T': 51, 'X': 32, 'A': 17, 'N': 12, 'Y': 1, 'Z': 3, 'L': 10, 'J': 14, 'G': 22, 'U': 11, 'R': 3, 'K': 12, 'W': 18, 'E': 7, 'B': 10, 'V': 3, 'Q': 1}
3
{'S': 28, 'M': 26, 'R': 36, 'V': 23, 'X': 30, 'E': 30, 'W': 27, 'P': 22, 'I': 43, 'Q': 10, 'K': 5, 'O': 4, 'F': 3, 'L': 21, 'H': 12, 'A': 10, 'G': 7, 'Z': 5, 'Y': 8, 'C': 5, 'J': 8, 'T': 8}
4
{'I': 19, 'U': 16, 'K': 38, 'V': 56, 'E': 35, 'F': 31, 'T': 9, 'Y': 16, 'B': 3, 'G': 9, 'J': 14, 'N': 11, 'R': 22, 'Z': 25, 'P': 9, 'W': 3, 'L': 12, 'M': 4, 'X': 7, 'D': 8, 'C': 20, 'S': 4}
5
{'F': 21, 'C': 43, 'H': 4

In [12]:
# Finding the letters with higher frequency
key_list = []
for i in range(0, n):
    temp = max(df_list[i].items(), key=operator.itemgetter(1))[0]
    print(temp + str(': ')+str(df_list[i][temp]))
    key_list.append(reverse_alphabet[cyclic_subtraction(alphabet[temp], 4)]) # We use 4 as E are the most frequent letter of english alphabet

# Taking the arithmetic form of the key's letters
print(key_list) # This is the key "EMPEROR"
for i in range(0,n):
    key_list[i] = alphabet[key_list[i]]
print(key_list)

I: 52
Q: 64
T: 51
I: 43
V: 56
S: 47
V: 49
['E', 'M', 'P', 'E', 'R', 'O', 'R']
[4, 12, 15, 4, 17, 14, 17]


In [13]:
''' Decrypting the message '''
# Knowing the key we should only use it backwards to decipher the message
decrypted_message = []
i=0 # this variable helps us to iterate the key_list in a cyclic way
for letter in coded_msg:
    if i == len(key_list):
        i=0
    # Decrypting the letters one by one
    # We transform the letters of the encrypted message to numbers, 
    # then we carry out the cyclic_subtraction and lastly we transform the result back to letters
    decrypted_message.append(reverse_alphabet[cyclic_subtraction(alphabet[letter], key_list[i])]) 
    i += 1
#print("".join(decrypted_message))

In [14]:
# The file will contain the decrypted message but without any punctuation marks or blank spaces
f = open("vigenere_decrypted.txt", "w")
f.write("".join(decrypted_message))
f.close()
# The corrected message is located in file "vigenere_decrypted_corrected.txt"